In [ ]:
!pip install psycopg2

In [ ]:
from urllib.parse import urlparse
import psycopg2
import psycopg2.extras
import os
from dotenv import load_dotenv
from pprint import pprint
load_dotenv()
url = urlparse(os.environ.get("postgres_url"))
connection = psycopg2.connect(
    host=url.hostname,
    port=url.port,
    database=url.path[1:],
    user=url.username,
    password=url.password
)
cursor = connection.cursor(cursor_factory = psycopg2.extras.RealDictCursor)

In [ ]:
query = """
select guilds_t.id, guilds_t.guild_name , message_count_t.message_count
from ( select messages_t.guild_id, count(messages_t.id) as message_count
from messages_t
group by messages_t.guild_id ) as message_count_t
join guilds_t on message_count_t.guild_id = guilds_t.id
order by message_count ASC;
"""
cursor.execute(query)
results_0 = cursor.fetchall()
pprint(results_0[:3])

In [ ]:
!pip install plotly
!pip install kaleido

In [ ]:
import plotly.io as pio
import plotly.express as px
pio.renderers.default = "svg"
fig = px.bar(x=["a", "b", "c"], y=[1, 3, 2])
fig.show()

In [ ]:
x_axis = []
y_axis = []
for guild in results_0:
    x_axis.append(guild["guild_name"])
    y_axis.append(guild["message_count"])

In [ ]:
import plotly.express as px
fig = px.bar(x=x_axis, y=y_axis)
fig.show()

In [ ]:
!pip install calmap

In [ ]:
biggest_guild = results_0[-1]
query = """
select distinct guilds_t.id , guilds_t.guild_name, month_timestamp, msg_count from (
	select
		distinct DATE_TRUNC('day', real_timestamp)
			         AS  month_timestamp,
	    COUNT(guild_id) AS msg_count,
	    guild_id 
	FROM messages_t
	GROUP BY guild_id, month_timestamp
) as month_messages_t
join guilds_t on month_messages_t.guild_id = guilds_t.id
where guilds_t.id = '%s'
order by guilds_t.id, month_timestamp;
"""
query = query % biggest_guild["id"]
print(query)

In [ ]:
cursor.execute(query)
print(query)
results_1 = cursor.fetchall()
pprint(results_1[:3])

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame.from_dict(results_1)
min_value = df['month_timestamp'].min()
max_value = df['month_timestamp'].max()
df['normalized_message_count'] = (df['month_timestamp'] - min_value) / (max_value - min_value)

In [ ]:
df.dtypes

In [ ]:
df

In [ ]:
import calmap

In [ ]:
import numpy as np; np.random.seed(sum(map(ord, 'calmap')))

In [ ]:
all_days = pd.date_range('03/05/2018', periods=2005, freq='D')
days = np.random.choice(all_days, 2005)
events = pd.Series(np.random.randn(len(days)), index=days)

In [ ]:
days.sort()

In [ ]:
days

In [ ]:
my_events = pd.Series(list( df['normalized_message_count'] ), index = days)

In [ ]:
my_events

In [ ]:
calmap.yearplot(my_events, year=2019)

In [ ]:
import matplotlib.font_manager

In [ ]:
calmap.calendarplot(
    my_events,
    monthticks=3,
    daylabels='MTWTFSS',
    dayticks=[0, 2, 4, 6],
    cmap='YlGn',
    fillcolor='grey',
    linewidth=0,
    fig_kws=dict(figsize=(20, 10)),
    yearlabels=True,
    yearascending=True
)

In [ ]:
df

In [ ]:
calmap.yearplot(series_df, year=2020)

In [ ]:
all_days = pd.date_range('1/15/2018', periods=265*6, freq='D')

In [ ]:
all_days = pd.date_range('1/15/2014', periods=700, freq='D')
days = np.random.choice(all_days, 500)
events = pd.Series(np.random.randn(len(days)), index=days)
calmap.yearplot(events, year=2015)

In [ ]:
len(days)